In [1]:
import sys, os
sys.path.append('..')
sys.path.append('../..')

In [2]:
import autograd
import pickle

In [3]:
import pystan

In [4]:
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy.stats.norm as norm

from viabel.vb import mean_field_t_variational_family, mean_field_gaussian_variational_family
from viabel.vb import make_stan_log_density, adagrad_optimize
from experiments import run_experiment

import seaborn as sns

sns.set_style('white')
sns.set_context('notebook', font_scale=2, rc={'lines.linewidth': 2})

In [5]:
from viabel.vb import  full_rank_gaussian_variational_family

In [6]:
from scipy.stats import t 
from itertools import product
from scipy.stats import t
from experiments import *

In [7]:
from viabel.vb import  rmsprop_IA_optimize_with_rhat, adam_IA_optimize_with_rhat

In [8]:
os.makedirs('../figures', exist_ok=True)

logtau_lim = [-2, 3.5]
mu_lim = [-5, 15]
theta1_lim = [-8, 22]

skip = 1 # how much to thin samples; larger values make the plots faster but let accurate

In [9]:
def plot_sample_contours(x_samples1, y_samples1, x_samples2, y_samples2, xlabel, ylabel, xlim, ylim, 
                    cmap1, cmap2, savepath=None, **kwargs):
    sns.kdeplot(x_samples1, y_samples1, cmap=cmap1, **kwargs)
    sns.kdeplot(x_samples2, y_samples2, cmap=cmap2, **kwargs)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xticks([])
    plt.yticks([])
    if savepath is not None:
        plt.savefig(savepath, bbox_inches='tight')
    plt.show()
    
def plot_sample_and_density_contours(x_samples, y_samples, logdensity, xlabel, ylabel, xlim, ylim, 
                    cmap_samples, cmap_density, savepath=None, **kwargs):
    sns.kdeplot(x_samples, y_samples, cmap=cmap_samples, **kwargs)
    x = np.linspace(*xlim, 100)
    y = np.linspace(*ylim, 100)
    X, Y = np.meshgrid(x, y)
    XY = np.concatenate([X[:,:,np.newaxis], Y[:,:,np.newaxis]], axis=2)
    Z = np.exp(logdensity(XY))
    plt.contour(X, Y, Z, cmap=cmap_density, linestyles='solid')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.xticks([])
    plt.yticks([])
    if savepath is not None:
        plt.savefig(savepath, bbox_inches='tight')
    plt.show()
    
def tranform_to_theta(ncp_samples):
    ncp_samples_tranformed = ncp_samples.copy()
    ncp_samples_tranformed[2:] = (ncp_samples_tranformed[0] 
                                  + np.exp(ncp_samples_tranformed[1]) * ncp_samples_tranformed[2:])
    return ncp_samples_tranformed

def get_ncp_approx_samples(var_family, opt_param, n_samples):
    ncp_samples = var_family.sample(opt_param, n_samples).T
    return ncp_samples, tranform_to_theta(ncp_samples)

In [10]:
n_params_cp=10
var_family_cp = mean_field_t_variational_family(n_params_cp, 40)
gaussian_mf_var_family_cp = mean_field_gaussian_variational_family(n_params_cp)
gaussian_fr_var_family_cp = full_rank_gaussian_variational_family(n_params_cp)


In [11]:
init_param_cp_fr = np.concatenate([np.ones(n_params_cp), .5*np.ones(int(n_params_cp*(n_params_cp+1)/2))])
gaussian_fr_var_family_cp.sample(init_param_cp_fr, n_samples=1)
init_param_cp_mf = np.concatenate([np.ones(n_params_cp), .5*np.ones(n_params_cp)])
a = gaussian_mf_var_family_cp.sample(init_param_cp_mf, n_samples=1)
#gaussian_mf_var_family_cp.logdensity(a, init_param_cp_mf) 
gaussian_fr_var_family_cp.logdensity(a, init_param_cp_fr) 

(55,)
(55, 1)


-82.71472820031981

In [12]:
try:
    eight_schools_cp_stan_model = pickle.load(open('eight_schools_cp1.pkl', 'rb'))
    #eight_schools_cp_stan_model = pystan.StanModel(file='eight_schools_cp.stan' ,
    #                                           model_name='eight_schools_cp')
except:
    eight_schools_cp_stan_model = pystan.StanModel(file='eight_schools_cp.stan', model_name='eight_schools_cp')
    with open('eight_schools_cp1.pkl', 'wb') as f:
        pickle.dump(eight_schools_cp_stan_model, f)
        

In [13]:
try:
    eight_schools_ncp_stan_model = pickle.load(open('eight_schools_ncp1.pkl', 'rb'))
    #eight_schools_ncp_stan_model = pystan.StanModel(file='eight_schools_ncp.stan' ,
    #                                           model_name='eight_schools_ncp')
except:
    eight_schools_ncp_stan_model = pystan.StanModel(file='eight_schools_ncp.stan', model_name='eight_schools_ncp')
    with open('eight_schools_ncp1.pkl', 'wb') as f:
        pickle.dump(eight_schools_ncp_stan_model, f)

In [14]:
# Data of the Eight Schools Model
J = 8
y = np.array([28.,  8., -3.,  7., -1.,  1., 18., 12.])
sigma = np.array([15., 10., 16., 11.,  9., 11., 10., 18.])
data = dict(J=J, y=y, sigma=sigma)

In [15]:
eight_schools_cp_fit = eight_schools_cp_stan_model.sampling(data=data, iter=11000, warmup=1000,
                                                            control=dict(adapt_delta=.99))

In [16]:
eight_schools_ncp_fit = eight_schools_ncp_stan_model.sampling(data=data, iter=32000, warmup=2000, thin=3,
                                                              control=dict(adapt_delta=.95))

In [17]:
eight_schools_ncp_fit

Inference for Stan model: eight_schools_ncp_ae46705580739ef95b05e742166c14cd.
4 chains, each with iter=32000; warmup=2000; thin=3; 
post-warmup draws per chain=10000, total post-warmup draws=40000.

                 mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu               4.36    0.02   3.31  -2.17   2.14   4.38   6.62   10.8  40318    1.0
tau              3.57    0.02   3.17   0.12   1.26   2.74   4.94  11.73  39389    1.0
theta_tilde[1]   0.32  5.0e-3   0.99  -1.63  -0.35   0.33    1.0   2.24  39891    1.0
theta_tilde[2]   0.11  4.7e-3   0.94  -1.75  -0.53   0.12   0.74   1.94  40195    1.0
theta_tilde[3]   -0.1  4.8e-3   0.97  -1.99  -0.75   -0.1   0.56    1.8  40422    1.0
theta_tilde[4]   0.06  4.7e-3   0.95  -1.81  -0.57   0.06    0.7   1.93  40130    1.0
theta_tilde[5]  -0.16  4.7e-3   0.93  -1.99  -0.78  -0.17   0.45    1.7  40235    1.0
theta_tilde[6]  -0.06  4.7e-3   0.95  -1.92  -0.69  -0.06   0.57   1.83  40687    1.0
theta_tilde[7]   0.36  4.8e

In [18]:
# number of parameters and parameter names in centered model
n_params_cp = len(eight_schools_cp_fit.constrained_param_names())
param_names_cp = ['mu', 'log_tau'] + eight_schools_cp_fit.flatnames[2:n_params_cp]

# number of parameters and parameter names in non-centered model
n_params_ncp = len(eight_schools_ncp_fit.constrained_param_names())
param_names_ncp = ['mu', 'log_tau'] + eight_schools_ncp_fit.flatnames[2:n_params_ncp]
param_names_ncp_transformed = ['mu', 'log_tau'] + eight_schools_ncp_fit.flatnames[n_params_ncp:]

# the centered and tranformed non-centered parameters should be the same
#np.testing.assert_array_equal(param_names_cp, param_names_ncp_transformed)

# construct matrix of samples (both original and transformed) from non-centered model 
samples_ncp_df = eight_schools_ncp_fit.to_dataframe(pars=eight_schools_ncp_fit.flatnames)
samples_ncp_df['log_tau'] = np.log(samples_ncp_df['tau'])
samples_ncp = samples_ncp_df.loc[:,param_names_ncp].values.T
samples_ncp_transformed = samples_ncp_df.loc[:,param_names_ncp_transformed].values.T

# use samples from non-centered model for ground true mean and covariance
true_mean_ncp = np.mean(samples_ncp, axis=1)
true_cov_ncp = np.cov(samples_ncp)
true_mean_ncp_tranformed = np.mean(samples_ncp_transformed, axis=1)
true_cov_ncp_tranformed = np.cov(samples_ncp_transformed)

In [19]:
eight_schools_cp_log_density = make_stan_log_density(eight_schools_cp_fit)

eight_schools_ncp_log_density = make_stan_log_density(eight_schools_ncp_fit)

In [20]:

var_family_cp = mean_field_t_variational_family(n_params_cp, 40)

init_param_cp = np.concatenate([true_mean_ncp_tranformed, .5*np.log(np.diag(true_cov_ncp_tranformed))])
klvi_cp, chivi_cp, klvi_cp_results, chivi_cp_results, other_klvi_cp_results, other_chivi_cp_results = \
    run_experiment(eight_schools_cp_log_density, var_family_cp, init_param_cp, 
                   true_mean_ncp_tranformed, true_cov_ncp_tranformed, 
                   learning_rate=.01, learning_rate_end=.001,
                   n_iters=10000, bound_w2=2500000, verbose=True)

|--------------|
|     KLVI     |
|--------------|


Average Loss = 8.5925: 100%|██████████| 10000/10000 [00:38<00:00, 260.09it/s]
/anaconda3/envs/viabel_env/lib/python3.6/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in log
  return f_raw(*args, **kwargs)
../viabel/optimization_diagnostics.py:148: RuntimeWarning: invalid value encountered in greater_equal
  real_idxs = weights >= 10 * np.finfo(float).eps
../viabel/optimization_diagnostics.py:161: RuntimeWarning: divide by zero encountered in double_scalars
  sigma = -k_post / b_post
/anaconda3/envs/viabel_env/lib/python3.6/site-packages/autograd/tracer.py:48: RuntimeWarning: invalid value encountered in log1p
  return f_raw(*args, **kwargs)
/anaconda3/envs/viabel_env/lib/python3.6/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in exp
  return f_raw(*args, **kwargs)


ValueError: too many values to unpack (expected 4)

In [ ]:
eight_schools_cp_log_density = make_stan_log_density(eight_schools_cp_fit)
t_var_family_cp = mean_field_t_variational_family(n_params_cp, 40)
mean_gauss_var_family_cp = mean_field_gaussian_variational_family(n_params_cp)
init_param_cp2 = np.concatenate([true_mean_ncp_tranformed, .5*np.log(np.diag(true_cov_ncp_tranformed))])
k=10
klvi_objective_and_grad = black_box_klvi(var_family_cp, eight_schools_cp_log_density, 100)
klvi_objective_and_grad_gaussian = black_box_klvi(mean_gauss_var_family_cp, eight_schools_ncp_log_density, 100)

In [ ]:
klvi_var_param_rms, avg_klvi_var_param_list_rms,_, klvi_history_rms, _, op_log_rms = \
    rmsprop_IA_optimize_with_rhat(30000, klvi_objective_and_grad_gaussian, init_param_cp2, k, learning_rate=.01, rhat_window=1000, n_optimisers=2)

In [ ]:
gauss_klvi_var_param_rms, gauss_avg_klvi_var_param_list_rms,_, gauss_klvi_history_rms, _, gauss_op_log_rms = \
    rmsprop_IA_optimize_with_rhat(30000, klvi_objective_and_grad, init_param_cp2, k, learning_rate=.01, rhat_window=1000, n_optimisers=2)

In [ ]:
plt.plot(klvi_history_rms[25000:30000])

In [ ]:
print(op_log_rms['r_hat_mean'])

print(op_log_rms['r_hat_sigma'])


In [ ]:
def cp_results_plot(other_results, method):
    if method not in ['klvi', 'chivi']:
        print('invalid method "{}"'.format(method))
        return
    cp_opt_param = other_results['opt_param']
    cp_mean, cp_log_scale = cp_opt_param[:n_params_cp], cp_opt_param[n_params_cp:]
    cp_log_density = lambda x: np.sum(t.logpdf(x, 40, cp_mean[np.newaxis,np.newaxis,1:3], 
                                               np.exp(cp_log_scale[np.newaxis,np.newaxis,1:3])), axis=-1)
    cmap2 = 'Reds' if method == 'klvi' else 'Blues'
    plot_sample_and_density_contours(
        np.log(eight_schools_ncp_fit['tau'][::skip]), eight_schools_ncp_fit['theta[1]'][::skip],
        cp_log_density, r'$\log(\tau)$', r'$\theta_1$', 
        logtau_lim, theta1_lim, 'Greys', cmap2,
        '../figures/8-schools-cp-log-tau-vs-theta1-{}.pdf'.format(method))

cp_results_plot(other_klvi_cp_results, 'klvi')
cp_results_plot(other_chivi_cp_results, 'chivi')